#Import Python Packages

In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on 6e08d5351d88 at 172.17.0.8 


:0: FutureWarning: IPython widgets are experimental and may change in the future.


#Make blank file-info sheet

Get files from database

In [10]:
files = dp.get_metatlas_files(experiment = '%hedlund%',name = '%hilic%',most_recent = True)

Save the files to an empty fileinfo sheet

In [9]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/hedlund_empty_fileinfo.tab',files)

#Create metatlas groups from filled in file-info sheet

In [18]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/FileInfoSheet_BHedlund_AllFiles_20160413 (1).txt',store=False)

#Create a new Atlas in metatlas from a spreadsheet

In [28]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/katherine_master_pos.txt',
                                       'master_atlas_pos_hilic_20160421',
                                       filetype='tab',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=False)

#Select groups of files to operate on

#Select Atlas to use

In [ ]:
atlas = metob.retrieve('Atlas',name = '%master%',username='*')
from datetime import datetime, date
for i,a in enumerate(atlas):
    print i, a.name,  datetime.utcfromtimestamp(a.last_modified)

In [ ]:
myAtlas = atlas[4]

#Get Data and Save it

In [ ]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
data = dp.get_data_for_groups_and_atlas(group,myAtlas,output_filename)

#Load a pre-existing datafile

In [7]:
my_file = '/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
project_label = '20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas'
# data = dp.get_data(my_file)

#Make Supplementary Tables

In [23]:

dp = reload(dp)

peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [28]:
dp = reload(dp)
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [9]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_files_each_compound')

7
74
nrows =  8
length of ymax is  5620


#Make Identification Figures

In [18]:
dp = reload(dp)
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

#Export Atlas To Spreadsheet

In [ ]:
dp.export_atlas_to_spreadsheet(myAtlas,output_filename)
